In [1]:
!pip3 install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162659 sha256=a0b43702741fdb650d4a6969f2843d0e0f27783b75fbd19013d384844600837e
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [11]:
import surprise
import pandas as pd
from surprise import Reader

In [3]:
from surprise import Dataset

movie_data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [15]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [16]:
!ls

drive  goodreads_ratings.csv  sample_data


In [17]:
book_ratings = pd.read_csv('goodreads_ratings.csv')
print(book_ratings.head())

                            user_id   book_id  \
0  d089c9b670c0b0b339353aebbace46a1   7686667   
1  6dcb2c16e12a41ae0c6c38e9d46f3292  18073066   
2  244e0ce681148a7586d7746676093ce9  13610986   
3  73fcc25ff29f8b73b3a7578aec846394  27274343   
4  f8880e158a163388a990b64fec7df300  11614718   

                          review_id  rating  \
0  3337e0e75701f7f682de11638ccdc60c       3   
1  7201aa3c1161f2bad81258b6d4686c16       5   
2  07a203f87bfe1b65ff58774667f6f80d       5   
3  8be2d87b07098c16f9742020ec459383       1   
4  a29c4ba03e33ad073a414ac775266c5f       4   

                                         review_text  \
0  Like Matched, this book felt like it was echoi...   
1  WOW again! 4,5 Stars \n So i wont forget to me...   
2  The second novel was hot & heavy. Not only in ...   
3  What a maddening waste of time. And I unfortun...   
4  4.5 stars! \n This was an awesome read! \n So ...   

                       date_added                    date_updated  \
0  Fri Apr 29 14

In [18]:
book_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       3500 non-null   object
 1   book_id       3500 non-null   int64 
 2   review_id     3500 non-null   object
 3   rating        3500 non-null   int64 
 4   review_text   3500 non-null   object
 5   date_added    3500 non-null   object
 6   date_updated  3500 non-null   object
 7   read_at       3167 non-null   object
 8   started_at    2395 non-null   object
 9   n_votes       3500 non-null   int64 
 10  n_comments    3500 non-null   int64 
dtypes: int64(4), object(7)
memory usage: 300.9+ KB


In [19]:
# Calculating the distribution of ratings
rating_distribution = book_ratings['rating'].value_counts().sort_index()

print(rating_distribution)


0     120
1     125
2     269
3     707
4    1278
5    1001
Name: rating, dtype: int64


In [20]:
# Filtering out ratings that are not in the range of 1 to 5
filtered_book_ratings = book_ratings[(book_ratings['rating'] >= 1) & (book_ratings['rating'] <= 5)]

print(filtered_book_ratings['rating'].value_counts().sort_index())


1     125
2     269
3     707
4    1278
5    1001
Name: rating, dtype: int64


In [21]:
from surprise import Reader

# Define the rating scale
rating_scale = (1, 5)

# Create the reader object
reader = Reader(rating_scale=rating_scale)



In [22]:

data = Dataset.load_from_df(filtered_book_ratings[['user_id', 'book_id', 'rating']], reader)



In [23]:
from surprise.model_selection import train_test_split


trainset, testset = train_test_split(data, test_size=0.2, random_state=7)



In [25]:
from surprise import KNNBasic
# Create the KNNBasic model
model = KNNBasic()

# Train the model on the training set
model.fit(trainset)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [26]:
# Example: predict the rating user with user_id "123" might give to item with item_id "456"
prediction = model.predict("123", "456")

# Print the estimated rating
print(prediction.est)


3.8250739644970415


In [27]:
predictions = model.test(testset)


In [28]:
from surprise import accuracy

# Calculate and print the RMSE
rmse = accuracy.rmse(predictions)


RMSE: 1.1105


In [29]:
# Assuming 'model' is your trained KNNBasic model
# User ID and Book ID as specified in the prompt
user_id = '8842281e1d1347389f2ab93d60773d4d'
book_id = '18007564'  # The Martian

# Predict the rating
prediction = model.predict(user_id, book_id)

# Print the estimated rating
print(f"Estimated rating for 'The Martian' by user {user_id}: {prediction.est}")


Estimated rating for 'The Martian' by user 8842281e1d1347389f2ab93d60773d4d: 3.8250739644970415


In [32]:
param_grid = {
    'n_factors': [50, 100, 150],  # Number of factors
    'n_epochs': [20, 30],  # Number of iterations
    'lr_all': [0.005, 0.01],  # Learning rate
    'reg_all': [0.02, 0.1]  # Regularization term
}


In [34]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data)


In [35]:
# Best RMSE score
print(f"Best RMSE score achieved: {gs.best_score['rmse']}")

# Parameters for the best RMSE score
print(f"Best parameters: {gs.best_params['rmse']}")


Best RMSE score achieved: 1.0500623452130438
Best parameters: {'n_factors': 150, 'n_epochs': 30, 'lr_all': 0.01, 'reg_all': 0.02}


In [36]:
best_model = SVD(**gs.best_params['rmse'])
# You can use cross_validate to train and evaluate the model on the entire dataset
cross_validate(best_model, data, measures=['RMSE'], cv=5, verbose=True)


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0689  1.0489  0.9895  1.0654  1.0786  1.0503  0.0319  
Fit time          0.08    0.08    0.09    0.08    0.08    0.08    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([1.06892437, 1.04887374, 0.98948647, 1.06539558, 1.07857449]),
 'fit_time': (0.08071422576904297,
  0.07972073554992676,
  0.08816361427307129,
  0.07747840881347656,
  0.08139848709106445),
 'test_time': (0.004189968109130859,
  0.0027878284454345703,
  0.0025751590728759766,
  0.0025055408477783203,
  0.0025196075439453125)}